In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../input/us-accidents/US_Accidents_Dec21_updated.csv')

In [3]:
df.dropna(inplace = True)
df=df[df.Severity != 0]
df.drop(columns = ['ID'], inplace = True)
df.drop(columns = ['Description'], inplace = True)

In [4]:
categorical_columns = df.select_dtypes(include=['object','bool','datetime64[ns]']).columns
categorical_columns
from sklearn import preprocessing

for i in categorical_columns:
    en = preprocessing.LabelEncoder()
    df[i]= en.fit_transform(df[i])

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
target = 'Severity'
y = df[target]
X = df.drop(target, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [7]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [8]:
inputs = tf.keras.Input(shape=(X.shape[1],))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

batch_size = 40
epochs = 100
import time
start = time.time()
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)
print("Total time: ", time.time() - start, "seconds")

2022-11-20 09:09:59.334911: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-11-20 09:09:59.654912: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
15094/15094 [==============================] - 28s 2ms/step - loss: 46.8165 - accuracy: 0.9280 - val_loss: 0.2738 - val_accuracy: 0.9436
Epoch 2/100
15094/15094 [==============================] - 27s 2ms/step - loss: 0.2771 - accuracy: 0.9433 - val_loss: 0.2736 - val_accuracy: 0.9436
Epoch 3/100
15094/15094 [==============================] - 27s 2ms/step - loss: 0.2747 - accuracy: 0.9433 - val_loss: 0.2740 - val_accuracy: 0.9436
Epoch 4/100
15094/15094 [==============================] - 27s 2ms/step - loss: 0.2747 - accuracy: 0.9433 - val_loss: 0.2737 - val_accuracy: 0.9436
Epoch 5/100
15094/15094 [==============================] - 27s 2ms/step - loss: 0.2746 - accuracy: 0.9433 - val_loss: 0.2738 - val_accuracy: 0.9436
Total time:  137.81362962722778 seconds


In [9]:
print("Test Accuracy:", model.evaluate(X_test, y_test, verbose=0)[1])

Test Accuracy: 0.9433755278587341
